In [114]:
import csv
from random import randint, choice

In [115]:
with open('lab6_dataset.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'second_name', 'salary', 'city', 'age', 'vacation_prefer', 'transport_prefer', 'target'])

In [116]:
def generate_data():
        name = choice(['John', 'Emily', 'Michael', 'Sophia', 'David', 'Emma', 'Matthew', 'Olivia', 'Daniel', 'Ava', 'Christopher', 'Isabella', 'Andrew', 'Mia', 'Ethan', 'Abigail', 'William', 'Madison', 'James', 'Sophie', 'Alexander', 'Grace', 'Nicholas', 'Avery', 'Benjamin'])
        second_name = choice(['Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson', 'Moore', 'Taylor', 'Anderson', 'Thomas', 'Jackson', 'White', 'Harris', 'Martin', 'Thompson', 'Garcia', 'Martinez', 'Lopez'])
        salary = randint(40000, 90000)
        city = choice(['Tokyo', 'Beijing', 'Sydney', 'Toronto', 'Berlin', 'Barcelona', 'Chicago', 'San Francisco', 'Seattle', 'Sydney', 'Dublin', 'Tokyo', 'Seoul', 'Cape Town', 'Mumbai', 'Singapore'])
        age = randint(25, 70)
        vacation_prefer = choice(['Safari', 'Mountain hiking', 'Cruise', 'Food tour', 'Historical sightseeing', 'Relaxation spa', 'Festival', 'Golf retreat'])
        transport_prefer = choice(['car', 'plane', 'train', 'cruise ship'])
        target = choice(['Tokyo', 'Paris', 'New York', 'Barcelona', 'Dubai'])

        return [name, second_name, salary, city, age, vacation_prefer, transport_prefer, target]


In [117]:
import pandas as pd

csv_file_path = 'lab6_dataset.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Name', 'Second Name', 'Salary', 'City', 'Age', 'Vacation Preference', 'Transport Preference', 'Target Destination'])
    for _ in range(1000):
        writer.writerow(generate_data())

print(f"CSV file '{csv_file_path}' has been generated with random data.")

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Use get_dummies to convert categorical columns to numerical format
df = pd.get_dummies(df, columns=['Name', 'Second Name', 'City', 'Vacation Preference', 'Transport Preference', 'Target Destination'], drop_first=True)

# Save the modified data to a new CSV file
modified_csv_path = 'lab6_dataset_numerical.csv'
df.to_csv(modified_csv_path, index=False)

print(f"Modified CSV file '{modified_csv_path}' has been generated with numerical data.")

CSV file 'lab6_dataset.csv' has been generated with random data.
Modified CSV file 'lab6_dataset_numerical.csv' has been generated with numerical data.


In [123]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('lab6_dataset.csv')

target_variable = df.columns.difference(['Name', 'Second Name', 'Salary', 'City', 'Age', 'Vacation Preference', 'Transport Preference', 'Target Destination'])

X = df.drop(target_variable, axis=1)
y = df[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [124]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

modified_csv_path = 'lab6_dataset_numerical.csv'
df = pd.read_csv(modified_csv_path)

target_variable = df.columns.difference(['Name', 'Second Name', 'Salary', 'City', 'Age', 'Vacation Preference', 'Transport Preference','Target Destination'])[0]

X = df.drop(target_variable, axis=1)
y = df[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, zero_division=1)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.93
Confusion Matrix:
[[185   0]
 [ 15   0]]
Classification Report:
              precision    recall  f1-score   support

       False       0.93      1.00      0.96       185
        True       1.00      0.00      0.00        15

    accuracy                           0.93       200
   macro avg       0.96      0.50      0.48       200
weighted avg       0.93      0.93      0.89       200



In [125]:
random_data = {'Name': ['John'],
               'Second Name': ['Smith'],
               'Salary': [65000],
               'City': ['New York'],
               'Age': [40],
               'Vacation Preference': ['Beach holiday'],
               'Transport Preference': ['plane']}

random_df = pd.DataFrame(random_data)

# Ensure dummy columns match those used during training
random_df = pd.get_dummies(random_df, columns=['Name', 'Second Name', 'City', 'Vacation Preference', 'Transport Preference'], drop_first=True)

missing_columns = set(X_train.columns) - set(random_df.columns)
for col in missing_columns:
    random_df[col] = 0

# Ensure the column order is the same as during training
random_df = random_df[X_train.columns]

# Make predictions
prediction = clf.predict(random_df)

print("Predicted Target Destination:", prediction[0])

Predicted Target Destination: False


In [126]:
from sklearn.model_selection import cross_val_score, GridSearchCV

X = df.drop(target_variable, axis=1)
y = df[target_variable]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)

cv_scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

Cross-Validation Scores: [0.94  0.935 0.935 0.935 0.935]
Mean CV Accuracy: 0.9359999999999999


In [127]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

print("Best Parameters from Grid Search:", grid_search.best_params_)
print("Best CV Accuracy from Grid Search:", grid_search.best_score_)


Best Parameters from Grid Search: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Best CV Accuracy from Grid Search: 0.9359999999999999


In [129]:
best_clf = RandomForestClassifier(random_state=42, **grid_search.best_params_)
best_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_clf.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, zero_division=1)

print("Accuracy: {0:.2f}".format(accuracy))
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.93
Confusion Matrix:
[[185   0]
 [ 15   0]]
Classification Report:
              precision    recall  f1-score   support

       False       0.93      1.00      0.96       185
        True       1.00      0.00      0.00        15

    accuracy                           0.93       200
   macro avg       0.96      0.50      0.48       200
weighted avg       0.93      0.93      0.89       200

